Imersão Dev Agentes de IA - Alura e Google Gemini
---

Aula 01 - Classificação de intenções com IA
---



In [36]:
!pip install -q --upgrade langchain langchain-google-genai google-generativeai

In [37]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI


GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')

In [38]:
# temperatura entre 0 e 1, 1 sendo criativo

In [39]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperatura=0.8,
    api_key=GOOGLE_API_KEY
)

In [40]:
resp_test = llm.invoke("Quem é você?")
print(resp_test.content)

Eu sou um modelo de linguagem grande, treinado pelo Google.


In [41]:
# Qual o objetivo do Agente?

In [42]:
TRIAGEM_PROMPT = (
    "Você é um gerador de README para perfil do GitHub. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_COMPLETAR" | "PEDIR_INFO" | "GERAR_BIO",\n'
    '  "carreira_academica": "ESTUDANTE" | "FORMADO/A" | "PÓS-GRADUADO/A",\n'
    '  "experiencia_profissional": "ESTAGIÁRIO/A" | "JÚNIOR" | "PLENO" | "SÊNIOR | "ESPECIALISTA",\n'
    '  "campos_faltantes: ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_COMPLETAR**: Perfil completo da pessoa, contendo nome, função, senioridade, hard skills e soft skills (Ex: "Meu nome é Maria, sou Especialista em TI Sênior, programadora javascript e autodidata", "Meu nome é José, sou estudante buscando estágio em TI, sei HMTL, CSS e JS").\n'
    '- **PEDIR_INFO**: Mensagem de apresentação vaga ou que faltam informações para identificar o perfil (Ex: "Sou estudante", "Trabalho com TI").\n'
    '- **GERAR_BIO**: Solicitação para criar Bio ou Perfil do GitHub com as informações da mensagem de forma completa e precisa."'
)

In [43]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

class TriagemOut(BaseModel):
    decisao: Literal["AUTO_COMPLETAR", "PEDIR_INFO", "GERAR_BIO"]
    carreira_academica: Literal["ESTUDANTE", "FORMADO/A", "PÓS-GRADUADO/A"]
    experiencia_profissional: Literal["ESTAGIÁRIO/A", "JÚNIOR", "PLENO", "SÊNIOR", "ESPECIALISTA"]
    campos_faltantes: List[str] = Field(default_factory=list)

In [44]:
llm_triagem = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.0,
    api_key=GOOGLE_API_KEY
)

In [45]:
from langchain_core.messages import SystemMessage, HumanMessage

triagem_chain = llm_triagem.with_structured_output(TriagemOut)

def triagem(mensagem: str) -> Dict:
    saida: TriagemOut = triagem_chain.invoke([
        SystemMessage(content=TRIAGEM_PROMPT),
        HumanMessage(content=mensagem)
    ])

    return saida.model_dump()

In [46]:
testes = ["Meu nome é Ana Maria sou Especialista de TI, Formada e conclui a Pós, sou Analista nível Pleno", "Meu Nome é Maria, sou estudante e estagiária"]

In [48]:
for msg_teste in testes:
    print(f"Pergunta: {msg_teste}\n -> Resposta: {triagem(msg_teste)}\n")

Pergunta: Meu nome é Ana Maria sou Especialista de TI, Formada e conclui a Pós, sou Analista nível Pleno
 -> Resposta: {'decisao': 'AUTO_COMPLETAR', 'carreira_academica': 'PÓS-GRADUADO/A', 'experiencia_profissional': 'PLENO', 'campos_faltantes': []}

Pergunta: Meu Nome é Maria, sou estudante e estagiária
 -> Resposta: {'decisao': 'AUTO_COMPLETAR', 'carreira_academica': 'ESTUDANTE', 'experiencia_profissional': 'ESTAGIÁRIO/A', 'campos_faltantes': []}

